In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [3]:
import json

with open('topic_testing/podcast_08.txt', 'rb') as f:
    request = json.load(f)

In [4]:
from main import handler

res = handler(request, None)
group = json.loads(res['body'])

[nltk_data] Downloading package wordnet to /tmp/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /tmp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-10-08T13:10:07.608229Z", "msg": "getting feature vector from mind service"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-10-08T13:10:07.609413Z", "msg": "Event before-parameter-build.lambda.Invoke: calling handler <function generate_idempotent_uuid at 0x7ff4faffac20>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-10-08T13:10:07.610751Z", "msg": "Event before-call.lambda.Invoke: calling handler <function inject_api_version_header_if_needed at 0x7ff4faffd680>"}
{"level": "debug", "filename": "endpoint.py", "lineno": 101, "module": "endpoint", "ts": "2019-10-08T13:10:07.611919Z", "msg": "Making request for OperationModel(name=Invoke) with params: {'url_path': '/2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations', 'query_string': {}, 'method': 'POST', 'headers': {'X-Amz-Invocation-Type': 'RequestRespon

{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-10-08T13:10:07.613723Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7ff4fad9ee10>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-10-08T13:10:07.615239Z", "msg": "Event choose-signer.lambda.Invoke: calling handler <function set_operation_specific_signer at 0x7ff4faffa170>"}
{"level": "debug", "filename": "auth.py", "lineno": 364, "module": "auth", "ts": "2019-10-08T13:10:07.668891Z", "msg": "Calculating signature using v4 auth."}
{"level": "debug", "filename": "auth.py", "lineno": 365, "module": "auth", "ts": "2019-10-08T13:10:07.669715Z", "msg": "CanonicalRequest:\nPOST\n/2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations\n\nhost:lambda.us-east-1.amazonaws.com\nx-amz-date:20191008T131007Z\nx-amz-invocation-type:RequestResponse\n\nhost;x-amz-

KeyboardInterrupt: 

In [ ]:
## visualization
import iso8601
from datetime import datetime
# meeting start time.
def formatTime(tz_time, datetime_object=False):
    isoTime = iso8601.parse_date(tz_time)
    ts = isoTime.timestamp()
    ts = datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S:%f")

    if datetime_object:
        ts = datetime.fromisoformat(ts)
    return ts

#m_time = formatTime("2019-09-19T06:05:00Z", True)
#m_time = formatTime("2019-09-22T09:37:00Z", True)
#m_time = formatTime("2019-09-16T09:53:21Z", True)
m_time = formatTime("2019-09-30T10:08:00Z", True)
for i in group['group'].keys():
    print ("\n\n\nPIMs ", i)
    print ("\n\nDiscussion:\n\n ")
    for seg in group['group'][i]:
        print ("Minutes from the start of the meeting: ", formatTime(seg['startTime'], True) - m_time , seg['id'],"\n")
        print (seg['originalText'],"\n")


In [ ]:
import boto3
from boto3 import client
from botocore.client import Config
import numpy as np
import json
aws_config = Config(
        connect_timeout=60,
        read_timeout=300,
        retries={"max_attempts": 0},
        region_name="us-east-1",
    )
lambda_client = client("lambda", config=aws_config)

def get_pims_score(req):

    #if req_data is None:
    #    lambda_payload = {"body": input_list}
    #    print (json.dumps(lambda_payload))
    #else:
    #    lambda_payload = {"body": {"request": req_data, "text_input": input_list}}
        
    try:
        #logger.info("Invoking lambda function")
        invoke_response = lambda_client.invoke(
            FunctionName="pim",
            InvocationType="RequestResponse",
            Payload=json.dumps(req),
        )
        lambda_output = (
            invoke_response["Payload"].read().decode("utf8")
        )
        response = json.loads(lambda_output)
        status_code = response["statusCode"]
        response_body = response["body"]

        if status_code == 200:
            result = json.loads(response_body)['d2vResult'][0]['distance']
        return result
    except Exception as e:
        print (e)
    return False

In [ ]:
pim_result = {}
pim_request = {"contextId": request["body"]["contextId"], "mindId": "01daaqyn9gbebc92aywnxedp0c"}
for seg in request['body']['segments']:
    pim_request["segments"] = [seg]
    # get_pims_score({"body":pim_request})
    pim_result[seg["recordingId"]] =  get_pims_score({"body":pim_request})

In [ ]:
group

In [ ]:
topic_pim = {
}
group_result = {}
for keys in group['group'].keys():
    for seg in group['group'][keys]:
        group_result[seg['recordingId']] = keys

In [ ]:
ranked_pims = sorted([(k,v) for (k,v) in pim_result.items()], key= lambda kv: kv[1])

In [ ]:
used_topics = []
group_no = None
index = 0
for (rec_id, distance) in ranked_pims:
    if rec_id in group_result.keys():
        group_no = group_result[rec_id]
        
        if group_no not in used_topics:
            topic_pim[index] = group_no
            used_topics.append(group_no)
            index += 1

In [120]:
topic_pim

{0: '24', 1: '40', 2: '32', 3: '15'}

In [125]:
final_output = []
final_output = list(map(lambda x: group['group'][x] , topic_pim.values()))

In [126]:
final_output[0][0]

{'id': '8ad2cdf53c0c47ee97ecc9eb649af405',
 'originalText': "Ux Designers have a a lot on their plate. They research that wire user tests the list goes on and off if you're new to the game this can all feel very overwhelming well throw that here out the window because we're about to embark on a journey through the sixteenth end of the UX design process that before starting any successful uX project you'll need to be in tune with two things your user in your brain. I start with the user big question around this is what problems are they making once you can identify the problem, then you can begin to start working to towards the solution second is understanding the brain and how does the project you're are working on aligned with the branch mission and goal to summarize in this stage you wanna identify the problems facing the users as well as the core goals of the brand and see how they along as a designer you probably have a gun of a*sumptions about what the user wants or doesn't want b

In [129]:
users = []
for result in final_output:
    temp_users = []
    for seg in result:
        if seg['spokenBy'] not in temp_users:
            temp_users.append(seg['spokenBy'])
    users.append(temp_users)

In [130]:
users

[['31a3ba4761854ad9a041ddf1c4c6a1dc'],
 ['31a3ba4761854ad9a041ddf1c4c6a1dc'],
 ['31a3ba4761854ad9a041ddf1c4c6a1dc'],
 ['31a3ba4761854ad9a041ddf1c4c6a1dc']]

# Testing topic level pims

## Read json Request

In [16]:
import json

with open('topic_testing/podcast_08.txt', 'rb') as f:
    request = json.load(f)

## Get Groups for the request

In [17]:
from main import handler

res = handler(request, None)
group = json.loads(res['body'])

{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-10-08T14:22:55.200769Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-10-08T14:22:55.201576Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 24, "module": "scorer", "ts": "2019-10-08T14:23:31.042747Z", "msg": "Request Sent"}
{"level": "info", "filename": "scorer.py", "lineno": 24, "module": "scorer", "ts": "2019-10-08T14:23:31.043755Z", "msg": "Request Sent"}
{"level": "info", "filename": "scorer.py", "lineno": 32, "module": "scorer", "ts": "2019-10-08T14:23:35.233715Z", "msg": "Response Recieved"}
{"level": "info", "filename": "scorer.py", "lineno": 32, "module": "scorer", "ts": "2019-10-08T14:23:35.234453Z", "msg": "Response Recieved"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 128, "module": "grouper_segments", "outlier thres

{"level": "info", "filename": "grouper_segments.py", "lineno": 128, "module": "grouper_segments", "outlier threshold is : ": 0.8663557167360496, "ts": "2019-10-08T14:23:40.370276Z", "msg": "Outlier Score"}
[(0, 0), (1, 0), (2, 0), (12, 0), (20, 0), (42, 0), (44, 0), (45, 0), (48, 0), (50, 0), (51, 0), (52, 0), (63, 0), (66, 0), (72, 0), (76, 0), (89, 0), (95, 0), (96, 0), (124, 0), (126, 0), (128, 0), (131, 0), (145, 0), (161, 0), (163, 0), (166, 0), (167, 0), (169, 0), (170, 0), (179, 0), (188, 0), (192, 0), (5, 1), (28, 1), (40, 1), (57, 1), (58, 1), (59, 1), (67, 1), (71, 1), (82, 1), (91, 1), (92, 1), (99, 1), (113, 1), (117, 1), (120, 1), (121, 1), (125, 1), (137, 1), (165, 1), (189, 1), (198, 1), (6, 2), (8, 2), (16, 2), (27, 2), (46, 2), (53, 2), (54, 2), (90, 2), (105, 2), (109, 2), (122, 2), (140, 2), (141, 2), (149, 2), (157, 2), (181, 2), (183, 2), (197, 2), (37, 3), (47, 4), (102, 5), (130, 6), (3, 7), (9, 7), (14, 7), (17, 7), (21, 7), (30, 7), (34, 7), (93, 7), (97, 7), (

cluster -=======>  22
('There are so many frameworks around which are providing first from challenges support to build enterprise sections.', '2019-10-08T12:15:03Z', 'fb52cb663aec4795aee38ccfd904d315', '8e6e0b2a5f6f4bbf92b829c13bc720f9')
cluster -=======>  23
('There is a lot of data in the world today generated not only by people but also by computers phones and other devices this will only continue to grow and the years call traditionally humans have alyzed data and adapted systems to the changes the patterns.', '2019-10-08T12:22:57Z', 'fb52cb663aec4795aee38ccfd904d315', '0512bcc3fcaa499a8979f9e3840ec705')
('The Toy has gotten quite good all you need is data developers and a willingness to take the punch for our purposes.', '2019-10-08T12:24:42Z', 'fb52cb663aec4795aee38ccfd904d315', '3c33ade92e514abdb065647430da28fc')
('It serves a useful purpose for us asume in particular we can split the definition into two parts using data and ask for questions.', '2019-10-08T12:24:42Z', 'fb52cb66

{"level": "info", "filename": "grouper_segments.py", "lineno": 469, "module": "grouper_segments", "PIMs": {"1": {"segment0": [["never, never, never develops any product. There are certain issues which are probably almost every time. Markers. Well, that problem is whenever you're designing a project, it was absolutely fine in your machine, the developer machine. But as soon as that product is being moved onto the production state, maybe onto servers or maybe somebody's else computer. Maybe your friends are maybe a manager. This computer. In that case, the product usually fails to work with the same performance or same optimization or the same level of working, even when that product has moved from one place to another place. Although here we're talking about the big skill products that include something like no django myfriend Javascript in all those stuffs just to give you a bare minimum. Basic example, whether you know of any bets site using PHP or maybe sp dot net. Whenever you work 

In [7]:
request

{'body': {'contextId': '01DBCR0S832GDJS7NGF9YW11WH',
  'mindId': '01daaqyn9gbebc92aywnxedp0c',
  'instanceId': 'd3e89f06-1ef6-486d-8b33-8d0f741ffdb1',
  'segments': [{'id': 'ec9f11b986a8413083e2816e15487eaf',
    'originalText': "I've been talking about. Why Dark net and what is darkness. This is a very interesting and that Spacy question which people need to understand about Microsoft Dark Knight framework.",
    'confidence': 0.840839393939394,
    'startTime': '2019-10-08T12:11:31Z',
    'endTime': '2019-10-08T12:11:47Z',
    'duration': 16,
    'recordingId': '90d37ecbf7564958905d74cbaed71a1a',
    'spokenBy': 'fb52cb663aec4795aee38ccfd904d315',
    'languageCode': 'en-IN',
    'transcriber': 'aws',
    'status': 'completed',
    'transcriptId': 'aa5051b7-ccda-42a0-aa39-4f10c215a7b5',
    'createdAt': '2019-10-08T12:11:52.17910052Z',
    'updatedAt': '2019-10-08T12:12:43.082891659Z',
    'deletedAt': None,
    'deleted': False},
   {'id': '965884b7479943de8c41e06dd5848e91',
    'or

## Get PIMs for the request

In [18]:
import boto3
from boto3 import client
from botocore.client import Config
import numpy as np
import json
aws_config = Config(
        connect_timeout=60,
        read_timeout=300,
        retries={"max_attempts": 0},
        region_name="us-east-1",
    )
lambda_client = client("lambda", config=aws_config)

def get_pims_score(req):

    #if req_data is None:
    #    lambda_payload = {"body": input_list}
    #    print (json.dumps(lambda_payload))
    #else:
    #    lambda_payload = {"body": {"request": req_data, "text_input": input_list}}
        
    try:
        #logger.info("Invoking lambda function")
        invoke_response = lambda_client.invoke(
            FunctionName="pim",
            InvocationType="RequestResponse",
            Payload=json.dumps(req),
        )
        lambda_output = (
            invoke_response["Payload"].read().decode("utf8")
        )
        response = json.loads(lambda_output)
        status_code = response["statusCode"]
        response_body = response["body"]

        if status_code == 200:
            result = json.loads(response_body)['d2vResult'][0]['distance']
        return result
    except Exception as e:
        print (e)
    return False

In [19]:
pim_result = {}
pim_response = {}
pim_request = {"contextId": request["body"]["contextId"], "mindId": "01DAAQY88QZB19JQZ5PRJFR76Y"}
pim_response = {"contextId": request["body"]["contextId"], "mindId": "01DAAQY88QZB19JQZ5PRJFR76Y", "segments": []}
temp = {}
for seg in request['body']['segments']:
    pim_request["segments"] = [seg]
    # get_pims_score({"body":pim_request})
    pim_result[seg["recordingId"]] =  get_pims_score({"body":pim_request})
    temp = seg
    temp["distance"] = pim_result[seg["recordingId"]]
    pim_response["segments"].append(temp)

## Extract topic level pims

In [20]:
import sys
sys.path.append("../")

import extract_topic_pims

In [21]:
from extract_topic_pims.main import handler

res = handler({"body":{"groups": group["group"], "pims": pim_response}}, None)
final_pims = json.loads(res)["body"]

{'707adb4c22d94bbaa87829add13473af': '4', 'd3dd3c881d6d48c98527f0cfa2fa07dd': '4', 'ef47783d7fc84638b03a9a48a58666a6': '1', '90d37ecbf7564958905d74cbaed71a1a': '1', 'b909a79f233c453394cf4f0a321c8911': '4', '98c6770c2216456fb27cd9388673196d': '4', '3cb7cb1291a2425496b6f8bf4f0a960f': '4', 'e8321c8fd0514a8eb5d4fc8f3c4009bc': '4', 'fd9abb69d46d4837948541d9dd56a71f': '4', '737bcc962bd24d118e33c319db9cdcad': '4', '34a368322892464bba69be90a0ba722b': '56', '9ff66cf02f1f4400bff244f9026f2815': '7', '0c0fb99e0b9440f5aff4f5370c30ca4c': '34', 'ada97e4118f746b398536feeb45627a0': '36', '2234236053514ddf92e1e3df76774fc5': '44', 'c6bd33d29a57456db25c64d3ba526ead': '40', '6abcc1be9429441dbce22f771f7623fb': '40', 'a5ba8f513aff44088bdc7d42bf12a431': '40', '2d7fc502b7aa4c7e9a41413607237858': '40', 'bec7953bf63c48bc9937947702495cea': '40', '1686ac7510f342ec824cfff3d806eea0': '40', 'a401bd726bee4c9bbd020cf0e34b29d2': '40', '6e8a0f76853e412a8a32f22360204386': '44', '45c16be51d2d49c59f8fdcbbf0a3b8e1': '56', '4

In [43]:
len(group['group']['76'])

KeyError: '76'

In [22]:
for groupid in final_pims:
    user_list =[]
    seg_list = []
    keyphrase = []
    for segi in groupid:
        if segi['spokenBy'] not in user_list:
            user_list.append(segi['spokenBy'])
        seg_list.append(segi['originalText'])
        #keyphrase.append(get_desc(segi['originalText']))
    #print ("User", end=" ")
    print (*[user_id_map[user] for user in user_list], sep=", ", end=" ")
    print ("Discussed \n\n Text: ", *seg_list, "\n\n")
    print (*get_desc(" ".join(sent for sent in seg_list))[1][:5], sep=", ")
    print ( "\n\n\n\n")

Reagan Rewop Discussed 

 Text:  everyone they share back again. Now, in this video, even talk about what is A.P.I. are also on this application. Programming Integration. Now, this is a very common Adam Bay you spent on. It is a problem in our society or a computer society. Begin zero f with people understand that some trick or not, we just wanted to be technically correct. So if you look up for the A.P.I. destination on any platform Wikipedia block, they are technically correct. That's one part, but they're not so much understandable. So in this video, we're going to make sure that you understand A.P.I. what is A.P.I.? And after watching this video, you never, ever watch any video on A.P.I. or block because you will be understanding each and every bit off idea. So let's get started. Let's talk about what is A.P.I.. Now go ahead, open up your favorite. Start changing and try to look up for What is it? Yeah, let me help you and try to read what is written on your very favorite Greek idi

watch any video on A. destination platform Wikipedia P. understand, remember the example of guy concept Docker install, access for a feature known sign of features pretty common, move ahead and talk project the product gon na about Docker fire ways, works on the developer machine my





Reagan Rewop Discussed 

 Text:  Microsoft doc*ment Framework in public and it evolved. Ideas into about Microsoft Chartered framework. Let me help you the why dot was fairly needed what purpose it is a strongly in industry if you know about C or programming languages or even then basic financial of these two from our languages C users has include S p I or downs to perform and Jeff and the scanning functionality. Similarly C does is as soon type of programming running language with object to date framework edit and it uses dot these two languages are almost same they perform same type of functionality, but they have to just in type of some time environment they have two different set of libraries on to

greet these container and containers publish, understand about Microsoft Dark Knight framework, project and move the product onto Web server, developer machine, PHP or maybe sp dot net





Reagan Rewop Discussed 

 Text:  World it is filled with data a lot of data pictures music words spreadsheets videos and it doesn't look like it's going to slow down any anytime soon machine learning brings the promise of driving me all of that partners further famously once said any sufficiently that technology is in this individual from that I found not to be magic rather and technology and you can utilize and answer questions with your data. This is cloud aI makers my name did you label well and each episode we will be exploring the art science and tools from machine learning along the way we'll see just how easy it is to create a meeting experiences and your valuable insights the value of machine learning is on just beginning to show itself. There's a lot of data in the world today generated not

In [12]:
import sys
sys.path.append("../../../ai-engine/pkg/")

from graphrank.core import GraphRank
from graphrank.utils import GraphUtils, TextPreprocess

gr = GraphRank()
tp = TextPreprocess()
gu = GraphUtils()

def get_desc(sentence):
    original_tokens, pos_tuple, filtered_pos_tuple = tp.preprocess_text(sentence, filter_by_pos=True, stop_words=False)
    word_graph = gr.build_word_graph(graph_obj=None, input_pos_text=pos_tuple, window=4, preserve_common_words=False)
    normal_keyphrase = gr.get_keyphrases(word_graph, pos_tuple, post_process=True)
    desc_keyphrase = gr.get_keyphrases(word_graph, pos_tuple, descriptive=True, post_process_descriptive=True)
    desc_keyphrase = sorted(desc_keyphrase, key=lambda kv:kv[1], reverse=True)
    normal_kp = [phrase for phrase, score in normal_keyphrase]
    desc_kp = [phrase for phrase, score in desc_keyphrase]
    
    return normal_kp, desc_kp

In [13]:
user_id_map = {}
user_id_map = {"b1e8787a9a1f4859ac11cbb6a8124fd9": "Venkata Dikshit", "fb52cb663aec4795aee38ccfd904d315":"Reagan Rewop", "81a3e15469374fceba1cf972faa209b2":"Arjun Kini", "ecfeeb757f0a4d47af1ebd513929264a":"Shubham"}


# testing hierarchy community

In [101]:
import json

with open('topic_testing/podcast_28.txt', 'rb') as f:
    request = json.load(f)

In [102]:
from main import handler

res = handler(request, None)
group = json.loads(res['body'])

{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T15:02:28.314253Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T15:02:28.315016Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T15:02:28.315649Z", "msg": "getting feature vector from mind service"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:02:28.316552Z", "msg": "Event before-parameter-build.lambda.Invoke: calling handler <function generate_idempotent_uuid at 0x7f97bfd1af80>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:02:28.317144Z", "msg": "Event before-parameter-build.lambda.Invoke: calling handler <function generate_idempotent_uuid at 0x7f97bfd1af80>"}
{"level": "debug", "filename"

{"level": "debug", "filename": "endpoint.py", "lineno": 101, "module": "endpoint", "ts": "2019-09-30T15:02:28.323420Z", "msg": "Making request for OperationModel(name=Invoke) with params: {'url_path': '/2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations', 'query_string': {}, 'method': 'POST', 'headers': {'X-Amz-Invocation-Type': 'RequestResponse', 'User-Agent': 'Boto3/1.9.197 Python/3.7.4 Linux/5.2.11-arch1-1-ARCH Botocore/1.12.197'}, 'body': b'{\"body\": \"{\\\\\"text\\\\\": [\\\\\"Indeed everyone they share back again with another video.\\\\\", \\\\\"Now in this video we will talk about what is a pi or also known as application programming interface.\\\\\", \\\\\"Now, this is a very common term very useful term.\\\\\", \\\\\"But there is a problem with our society or our computer Society be give XnumberXfXnumberX, whether people understand that subject or not.\\\\\", \\\\\"We just want it to be technically correct.\\\\\", \\\\\"So if you look up for the API definition o

{"level": "debug", "filename": "endpoint.py", "lineno": 101, "module": "endpoint", "ts": "2019-09-30T15:02:28.325038Z", "msg": "Making request for OperationModel(name=Invoke) with params: {'url_path': '/2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations', 'query_string': {}, 'method': 'POST', 'headers': {'X-Amz-Invocation-Type': 'RequestResponse', 'User-Agent': 'Boto3/1.9.197 Python/3.7.4 Linux/5.2.11-arch1-1-ARCH Botocore/1.12.197'}, 'body': b'{\"body\": \"{\\\\\"text\\\\\": [\\\\\"Indeed everyone they share back again with another video.\\\\\", \\\\\"Now in this video we will talk about what is a pi or also known as application programming interface.\\\\\", \\\\\"Now, this is a very common term very useful term.\\\\\", \\\\\"But there is a problem with our society or our computer Society be give XnumberXfXnumberX, whether people understand that subject or not.\\\\\", \\\\\"We just want it to be technically correct.\\\\\", \\\\\"So if you look up for the API definition o

{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:02:28.326537Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:02:28.327136Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:02:28.327728Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:02:28.328450Z", "msg": "Event choose-signer.lambda.Invoke: calling handler <f

{"level": "debug", "filename": "connectionpool.py", "lineno": 396, "module": "connectionpool", "ts": "2019-09-30T15:04:06.271674Z", "msg": "https://lambda.us-east-1.amazonaws.com:443 \"POST /2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations HTTP/1.1\" 200 4595271"}
{"level": "debug", "filename": "connectionpool.py", "lineno": 396, "module": "connectionpool", "ts": "2019-09-30T15:04:06.272984Z", "msg": "https://lambda.us-east-1.amazonaws.com:443 \"POST /2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations HTTP/1.1\" 200 4595271"}
{"level": "debug", "filename": "connectionpool.py", "lineno": 396, "module": "connectionpool", "ts": "2019-09-30T15:04:06.273614Z", "msg": "https://lambda.us-east-1.amazonaws.com:443 \"POST /2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations HTTP/1.1\" 200 4595271"}
{"level": "debug", "filename": "parsers.py", "lineno": 234, "module": "parsers", "ts": "2019-09-30T15:04:06.274688Z", "msg": "Response headers: {'Date': 'Mon

{"level": "info", "filename": "grouper_segments.py", "lineno": 419, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["Everyone they share back again with another video. Now in this video. We will talk about what is a pi or also known as Catholic."], "2019-09-28T05:34:40Z", "31a3ba4761854ad9a041ddf1c4c6a1dc", "4e865bf0f59b440db31c432cf51347b7"], "segment1": [["Indeed everyone they share back again with another video. Now in this video we will talk about what is a pi or also known as application programming interface. Now, this is a very common term very useful term. But there is a problem with our society or our computer Society be give 0f2, whether people understand that subject or not. We just want it to be technically correct. So if you look up for the API definition on any platform Wikipedia blog, they are technically correct. That's one part when they are not so much understandable. So in this video we are going to make sure that you understand and API what else API and a

{"level": "info", "filename": "grouper_segments.py", "lineno": 419, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["Everyone they share back again with another video. Now in this video. We will talk about what is a pi or also known as Catholic."], "2019-09-28T05:34:40Z", "31a3ba4761854ad9a041ddf1c4c6a1dc", "4e865bf0f59b440db31c432cf51347b7"], "segment1": [["Indeed everyone they share back again with another video. Now in this video we will talk about what is a pi or also known as application programming interface. Now, this is a very common term very useful term. But there is a problem with our society or our computer Society be give 0f2, whether people understand that subject or not. We just want it to be technically correct. So if you look up for the API definition on any platform Wikipedia blog, they are technically correct. That's one part when they are not so much understandable. So in this video we are going to make sure that you understand and API what else API and a

In [105]:
## visualization
import iso8601
from datetime import datetime
# meeting start time.
def formatTime(tz_time, datetime_object=False):
    isoTime = iso8601.parse_date(tz_time)
    ts = isoTime.timestamp()
    ts = datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S:%f")

    if datetime_object:
        ts = datetime.fromisoformat(ts)
    return ts

#m_time = formatTime("2019-09-19T06:05:00Z", True)
#m_time = formatTime("2019-09-22T09:37:00Z", True)
m_time = formatTime("2019-09-28T05:34:21Z", True)
#m_time = formatTime("2019-09-30T10:28:00Z", True)
for i in group['group'].keys():
    print ("\n\n\nPIMs ", i)
    print ("\n\nDiscussion:\n\n ")
    for seg in group['group'][i]:
        print ("Minutes from the start of the meeting: ", formatTime(seg['startTime'], True) - m_time , seg['id'],"\n")
        print (seg['originalText'],"\n")





PIMs  0


Discussion:

 
Minutes from the start of the meeting:  0:00:19 4e865bf0f59b440db31c432cf51347b7 

Everyone they share back again with another video. Now in this video. We will talk about what is a pi or also known as Catholic. 

Minutes from the start of the meeting:  0:00:58 a0349078dbc746199b273b5a3b3e8f51 

Indeed everyone they share back again with another video. Now in this video we will talk about what is a pi or also known as application programming interface. Now, this is a very common term very useful term. But there is a problem with our society or our computer Society be give 0f2, whether people understand that subject or not. We just want it to be technically correct. So if you look up for the API definition on any platform Wikipedia blog, they are technically correct. That's one part when they are not so much understandable. So in this video we are going to make sure that you understand and API what else API and after watching this video you never ever watch a

In [106]:
groups = []
for i in group['group'].keys():
    if len(group['group'][i])==1:
        continue
    else:
        temp = []
        for seg in group['group'][i]:
            temp.append(seg['originalText'])
        groups.append(' '.join(temp))

In [107]:
group_result = {}
#group_response = {}
group_request = {"contextId": request["body"]["contextId"], "mindId": "01daaqyn9gbebc92aywnxedp0c", "instanceId": "xyz"}
#group_response = {"contextId": request["body"]["contextId"], "mindId": "01DADP74WFV607KNPCB6VVXGTG", "segments": []}
temp = {}
group_itr = None
for segments_id in group['group'].keys():
    if len(group['group'][segments_id]) > 2:
        group_request['segments'] = group['group'][segments_id]
        res = handler({"body":group_request}, None)
        group_itr = json.loads(res['body'])
        break

{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T15:16:21.833136Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T15:16:21.833949Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T15:16:21.834604Z", "msg": "getting feature vector from mind service"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:16:21.835641Z", "msg": "Event before-parameter-build.lambda.Invoke: calling handler <function generate_idempotent_uuid at 0x7f97bfd1af80>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:16:21.836270Z", "msg": "Event before-parameter-build.lambda.Invoke: calling handler <function generate_idempotent_uuid at 0x7f97bfd1af80>"}
{"level": "debug", "filename"

{"level": "debug", "filename": "endpoint.py", "lineno": 101, "module": "endpoint", "ts": "2019-09-30T15:16:21.841634Z", "msg": "Making request for OperationModel(name=Invoke) with params: {'url_path': '/2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations', 'query_string': {}, 'method': 'POST', 'headers': {'X-Amz-Invocation-Type': 'RequestResponse', 'User-Agent': 'Boto3/1.9.197 Python/3.7.4 Linux/5.2.11-arch1-1-ARCH Botocore/1.12.197'}, 'body': b'{\"body\": \"{\\\\\"text\\\\\": [\\\\\"Everyone they share back again with another video.\\\\\", \\\\\"We will talk about what is a pi or also known as Catholic.\\\\\", \\\\\"Indeed everyone they share back again with another video.\\\\\", \\\\\"Now in this video we will talk about what is a pi or also known as application programming interface.\\\\\", \\\\\"Now, this is a very common term very useful term.\\\\\", \\\\\"But there is a problem with our society or our computer Society be give XnumberXfXnumberX, whether people underst

{"level": "debug", "filename": "endpoint.py", "lineno": 101, "module": "endpoint", "ts": "2019-09-30T15:16:21.842960Z", "msg": "Making request for OperationModel(name=Invoke) with params: {'url_path': '/2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations', 'query_string': {}, 'method': 'POST', 'headers': {'X-Amz-Invocation-Type': 'RequestResponse', 'User-Agent': 'Boto3/1.9.197 Python/3.7.4 Linux/5.2.11-arch1-1-ARCH Botocore/1.12.197'}, 'body': b'{\"body\": \"{\\\\\"text\\\\\": [\\\\\"Everyone they share back again with another video.\\\\\", \\\\\"We will talk about what is a pi or also known as Catholic.\\\\\", \\\\\"Indeed everyone they share back again with another video.\\\\\", \\\\\"Now in this video we will talk about what is a pi or also known as application programming interface.\\\\\", \\\\\"Now, this is a very common term very useful term.\\\\\", \\\\\"But there is a problem with our society or our computer Society be give XnumberXfXnumberX, whether people underst

{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:16:21.844338Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:16:21.845103Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:16:21.845809Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:16:21.846798Z", "msg": "Event choose-signer.lambda.Invoke: calling handler <f

{"level": "debug", "filename": "connectionpool.py", "lineno": 396, "module": "connectionpool", "ts": "2019-09-30T15:17:31.608069Z", "msg": "https://lambda.us-east-1.amazonaws.com:443 \"POST /2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations HTTP/1.1\" 200 2153774"}
{"level": "debug", "filename": "parsers.py", "lineno": 234, "module": "parsers", "ts": "2019-09-30T15:17:31.609441Z", "msg": "Response headers: {'Date': 'Mon, 30 Sep 2019 15:18:13 GMT', 'Content-Type': 'application/json', 'Content-Length': '2153774', 'Connection': 'keep-alive', 'x-amzn-RequestId': '326d4a56-be3d-4ab5-8724-72d3bb2f89af', 'x-amzn-Remapped-Content-Length': '0', 'X-Amz-Executed-Version': '$LATEST', 'X-Amzn-Trace-Id': 'root=1-5d921c6f-7cdeec60f9fc47486cd2f448;sampled=0'}"}
{"level": "debug", "filename": "parsers.py", "lineno": 234, "module": "parsers", "ts": "2019-09-30T15:17:31.610493Z", "msg": "Response headers: {'Date': 'Mon, 30 Sep 2019 15:18:13 GMT', 'Content-Type': 'application/json', 'Conten

{"level": "info", "filename": "grouper_segments.py", "lineno": 419, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["Everyone they share back again with another video. Now in this video. We will talk about what is a pi or also known as Catholic."], "2019-09-28T05:34:40Z", "31a3ba4761854ad9a041ddf1c4c6a1dc", "4e865bf0f59b440db31c432cf51347b7"], "segment1": [["Indeed everyone they share back again with another video. Now in this video we will talk about what is a pi or also known as application programming interface. Now, this is a very common term very useful term. But there is a problem with our society or our computer Society be give 0f2, whether people understand that subject or not. We just want it to be technically correct. So if you look up for the API definition on any platform Wikipedia blog, they are technically correct. That's one part when they are not so much understandable. So in this video we are going to make sure that you understand and API what else API and a

{"level": "info", "filename": "grouper_segments.py", "lineno": 419, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["Everyone they share back again with another video. Now in this video. We will talk about what is a pi or also known as Catholic."], "2019-09-28T05:34:40Z", "31a3ba4761854ad9a041ddf1c4c6a1dc", "4e865bf0f59b440db31c432cf51347b7"], "segment1": [["Indeed everyone they share back again with another video. Now in this video we will talk about what is a pi or also known as application programming interface. Now, this is a very common term very useful term. But there is a problem with our society or our computer Society be give 0f2, whether people understand that subject or not. We just want it to be technically correct. So if you look up for the API definition on any platform Wikipedia blog, they are technically correct. That's one part when they are not so much understandable. So in this video we are going to make sure that you understand and API what else API and a

In [108]:
## visualization
import iso8601
from datetime import datetime
# meeting start time.
def formatTime(tz_time, datetime_object=False):
    isoTime = iso8601.parse_date(tz_time)
    ts = isoTime.timestamp()
    ts = datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S:%f")

    if datetime_object:
        ts = datetime.fromisoformat(ts)
    return ts

#m_time = formatTime("2019-09-19T06:05:00Z", True)
#m_time = formatTime("2019-09-22T09:37:00Z", True)
#m_time = formatTime("2019-09-16T09:53:21Z", True)
m_time = formatTime("2019-09-30T10:28:00Z", True)
m_time = formatTime("2019-09-28T05:34:21Z", True)
for i in group_itr['group'].keys():
    print ("\n\n\nPIMs ", i)
    print ("\n\nDiscussion:\n\n ")
    for seg in group_itr['group'][i]:
        print ("Minutes from the start of the meeting: ", formatTime(seg['startTime'], True) - m_time , seg['id'],"\n")
        print (seg['originalText'],"\n")





PIMs  0


Discussion:

 
Minutes from the start of the meeting:  0:00:19 4e865bf0f59b440db31c432cf51347b7 

Everyone they share back again with another video. Now in this video. We will talk about what is a pi or also known as Catholic. 

Minutes from the start of the meeting:  0:00:58 a0349078dbc746199b273b5a3b3e8f51 

Indeed everyone they share back again with another video. Now in this video we will talk about what is a pi or also known as application programming interface. Now, this is a very common term very useful term. But there is a problem with our society or our computer Society be give 0f2, whether people understand that subject or not. We just want it to be technically correct. So if you look up for the API definition on any platform Wikipedia blog, they are technically correct. That's one part when they are not so much understandable. So in this video we are going to make sure that you understand and API what else API and after watching this video you never ever watch a

In [89]:
group_result = {}
#group_response = {}
group_request = {"contextId": request["body"]["contextId"], "mindId": "01DADP74WFV607KNPCB6VVXGTG", "instanceId": "xyz"}
#group_response = {"contextId": request["body"]["contextId"], "mindId": "01DADP74WFV607KNPCB6VVXGTG", "segments": []}
temp = {}
group_itr_2 = None
for segments_id in group_itr['group'].keys():
    if len(group_itr['group'][segments_id]) > 2:
        group_request['segments'] = group_itr['group'][segments_id]
        res = handler({"body":group_request}, None)
        group_itr_2 = json.loads(res['body'])
        break

{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T14:29:07.140392Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T14:29:07.141156Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T14:29:07.142037Z", "msg": "getting feature vector from mind service"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T14:29:07.142868Z", "msg": "Event before-parameter-build.lambda.Invoke: calling handler <function generate_idempotent_uuid at 0x7f97bfd1af80>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T14:29:07.143833Z", "msg": "Event before-parameter-build.lambda.Invoke: calling handler <function generate_idempotent_uuid at 0x7f97bfd1af80>"}
{"level": "debug", "filename"

{"level": "debug", "filename": "endpoint.py", "lineno": 101, "module": "endpoint", "ts": "2019-09-30T14:29:07.149214Z", "msg": "Making request for OperationModel(name=Invoke) with params: {'url_path': '/2015-03-31/functions/mind-01dadp74wfv607knpcb6vvxgtg/invocations', 'query_string': {}, 'method': 'POST', 'headers': {'X-Amz-Invocation-Type': 'RequestResponse', 'User-Agent': 'Boto3/1.9.197 Python/3.7.4 Linux/5.2.11-arch1-1-ARCH Botocore/1.12.197'}, 'body': b'{\"body\": \"{\\\\\"text\\\\\": \\\\\" Okay I think my join let us just started with that community on the language models right yep so. Dont how any with having but engine General communities. Yeah, it is was giving good results. I just few meetings so in that I also observe that we are able to say generated good groups are coming up and the general call the development. The segments go with instances as code low so by was paying outfit with as a sentence level and I group the scale of four changes but generally does not behavior.

{"level": "debug", "filename": "endpoint.py", "lineno": 101, "module": "endpoint", "ts": "2019-09-30T14:29:07.150331Z", "msg": "Making request for OperationModel(name=Invoke) with params: {'url_path': '/2015-03-31/functions/mind-01dadp74wfv607knpcb6vvxgtg/invocations', 'query_string': {}, 'method': 'POST', 'headers': {'X-Amz-Invocation-Type': 'RequestResponse', 'User-Agent': 'Boto3/1.9.197 Python/3.7.4 Linux/5.2.11-arch1-1-ARCH Botocore/1.12.197'}, 'body': b'{\"body\": \"{\\\\\"text\\\\\": \\\\\" Okay I think my join let us just started with that community on the language models right yep so. Dont how any with having but engine General communities. Yeah, it is was giving good results. I just few meetings so in that I also observe that we are able to say generated good groups are coming up and the general call the development. The segments go with instances as code low so by was paying outfit with as a sentence level and I group the scale of four changes but generally does not behavior.

{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T14:29:07.151367Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T14:29:07.152272Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T14:29:07.152928Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T14:29:07.153688Z", "msg": "Event choose-signer.lambda.Invoke: calling handler <f

{"level": "debug", "filename": "connectionpool.py", "lineno": 396, "module": "connectionpool", "ts": "2019-09-30T14:29:35.851920Z", "msg": "https://lambda.us-east-1.amazonaws.com:443 \"POST /2015-03-31/functions/mind-01dadp74wfv607knpcb6vvxgtg/invocations HTTP/1.1\" 200 3042138"}
{"level": "debug", "filename": "parsers.py", "lineno": 234, "module": "parsers", "ts": "2019-09-30T14:29:35.853292Z", "msg": "Response headers: {'Date': 'Mon, 30 Sep 2019 14:30:17 GMT', 'Content-Type': 'application/json', 'Content-Length': '3042138', 'Connection': 'keep-alive', 'x-amzn-RequestId': '4eb66edf-ff05-40af-9613-4a7d8e93c0d7', 'x-amzn-Remapped-Content-Length': '0', 'X-Amz-Executed-Version': '$LATEST', 'X-Amzn-Trace-Id': 'root=1-5d92115d-b957f7527c2b1c3c87c74ea0;sampled=0'}"}
{"level": "debug", "filename": "parsers.py", "lineno": 234, "module": "parsers", "ts": "2019-09-30T14:29:35.855192Z", "msg": "Response headers: {'Date': 'Mon, 30 Sep 2019 14:30:17 GMT', 'Content-Type': 'application/json', 'Conten

{"level": "info", "filename": "grouper_segments.py", "lineno": 419, "module": "grouper_segments", "PIMs": {"3": {"segment0": [["Okay I think my join let's just started with that community on the language models right yep so."], "2019-09-30T10:29:13Z", "b1e8787a9a1f4859ac11cbb6a8124fd9", "ecc525a1a4d34f53aeaf2d97feada15e"], "segment1": [["Don't how any with having but engine General communities."], "2019-09-30T10:29:30Z", "b1e8787a9a1f4859ac11cbb6a8124fd9", "069ed6f0a2434222a5fc87c90c6dbe28"], "segment2": [["Yeah, it's was giving good results. I just few meetings so in that I also observe that we are able to say generated good groups are coming up and the general call the development. The segments go with instances as code low so by was paying outfit with as a sentence level and I group the scale of four changes but generally doesn't behavior. Yeah."], "2019-09-30T10:29:49Z", "ecfeeb757f0a4d47af1ebd513929264a", "74cf907c8c1d473abb406d002d35c3a9"], "segment3": [["Brand able to distinguis

{"level": "info", "filename": "grouper_segments.py", "lineno": 419, "module": "grouper_segments", "PIMs": {"3": {"segment0": [["Okay I think my join let's just started with that community on the language models right yep so."], "2019-09-30T10:29:13Z", "b1e8787a9a1f4859ac11cbb6a8124fd9", "ecc525a1a4d34f53aeaf2d97feada15e"], "segment1": [["Don't how any with having but engine General communities."], "2019-09-30T10:29:30Z", "b1e8787a9a1f4859ac11cbb6a8124fd9", "069ed6f0a2434222a5fc87c90c6dbe28"], "segment2": [["Yeah, it's was giving good results. I just few meetings so in that I also observe that we are able to say generated good groups are coming up and the general call the development. The segments go with instances as code low so by was paying outfit with as a sentence level and I group the scale of four changes but generally doesn't behavior. Yeah."], "2019-09-30T10:29:49Z", "ecfeeb757f0a4d47af1ebd513929264a", "74cf907c8c1d473abb406d002d35c3a9"], "segment3": [["Brand able to distinguis

In [90]:
## visualization
import iso8601
from datetime import datetime
# meeting start time.
def formatTime(tz_time, datetime_object=False):
    isoTime = iso8601.parse_date(tz_time)
    ts = isoTime.timestamp()
    ts = datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S:%f")

    if datetime_object:
        ts = datetime.fromisoformat(ts)
    return ts

#m_time = formatTime("2019-09-19T06:05:00Z", True)
#m_time = formatTime("2019-09-22T09:37:00Z", True)
#m_time = formatTime("2019-09-16T09:53:21Z", True)
m_time = formatTime("2019-09-30T10:28:00Z", True)
for i in group_itr_2['group'].keys():
    print ("\n\n\nPIMs ", i)
    print ("\n\nDiscussion:\n\n ")
    for seg in group_itr_2['group'][i]:
        print ("Minutes from the start of the meeting: ", formatTime(seg['startTime'], True) - m_time , seg['id'],"\n")
        print (seg['originalText'],"\n")





PIMs  3


Discussion:

 
Minutes from the start of the meeting:  0:21:13 ecc525a1a4d34f53aeaf2d97feada15e 

Okay I think my join let's just started with that community on the language models right yep so. 

Minutes from the start of the meeting:  0:21:30 069ed6f0a2434222a5fc87c90c6dbe28 

Don't how any with having but engine General communities. 

Minutes from the start of the meeting:  0:21:49 74cf907c8c1d473abb406d002d35c3a9 

Yeah, it's was giving good results. I just few meetings so in that I also observe that we are able to say generated good groups are coming up and the general call the development. The segments go with instances as code low so by was paying outfit with as a sentence level and I group the scale of four changes but generally doesn't behavior. Yeah. 

Minutes from the start of the meeting:  0:22:27 2c1b3cd9bfde44af956e7c0c7b3716fd 

Brand able to distinguish domain out of the and that case. 

Minutes from the start of the meeting:  0:22:33 78492bd0a4eb4de1810c98

In [75]:
## visualization
import iso8601
from datetime import datetime
# meeting start time.
def formatTime(tz_time, datetime_object=False):
    isoTime = iso8601.parse_date(tz_time)
    ts = isoTime.timestamp()
    ts = datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S:%f")

    if datetime_object:
        ts = datetime.fromisoformat(ts)
    return ts

#m_time = formatTime("2019-09-19T06:05:00Z", True)
#m_time = formatTime("2019-09-22T09:37:00Z", True)
#m_time = formatTime("2019-09-16T09:53:21Z", True)
m_time = formatTime("2019-09-30T10:08:00Z", True)
for i in group['group'].keys():
    print ("\n\n\nPIMs ", i)
    print ("\n\nDiscussion:\n\n ")
    for seg in group['group'][i]:
        print ("Minutes from the start of the meeting: ", formatTime(seg['startTime'], True) - m_time , seg['id'],"\n")
        print (seg['originalText'],"\n")





PIMs  0


Discussion:

 
Minutes from the start of the meeting:  0:21:13 ecc525a1a4d34f53aeaf2d97feada15e 

Okay I think my join let's just started with that community on the language models right yep so. 

Minutes from the start of the meeting:  0:21:30 069ed6f0a2434222a5fc87c90c6dbe28 

Don't how any with having but engine General communities. 

Minutes from the start of the meeting:  0:21:49 74cf907c8c1d473abb406d002d35c3a9 

Yeah, it's was giving good results. I just few meetings so in that I also observe that we are able to say generated good groups are coming up and the general call the development. The segments go with instances as code low so by was paying outfit with as a sentence level and I group the scale of four changes but generally doesn't behavior. Yeah. 

Minutes from the start of the meeting:  0:22:27 2c1b3cd9bfde44af956e7c0c7b3716fd 

Brand able to distinguish domain out of the and that case. 

Minutes from the start of the meeting:  0:22:33 78492bd0a4eb4de1810c98